Generative AI with LSTM - Text Generation


In [ ]:

import requests
import re
import numpy as np
import tensorflow as tf
import os
import time

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.layers import Input
import pickle

# -------------------------1. Load Dataset -------------------------
print("="*60)
print("Loading dataset from Project Gutenberg...")
print("="*60)

url = "https://www.gutenberg.org/files/100/100-0.txt"
try:
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    text = response.text.lower()
    print("✓ Dataset downloaded successfully")
except Exception as e:
    print(f"✗ Error downloading: {e}")
    exit(1)

start = text.find("the complete works of william shakespeare")
end = text.find("end of the project gutenberg ebook")

if start == -1 or end == -1:
    print("Warning: Could not find expected markers, using first 1M characters")
    start = 0
    end = 1000000

text = text[start:end]


text = text[:500000]
print(f"Using optimized dataset: {len(text):,} characters")


text = re.sub(r"[^a-z\s.,!?;:]", "", text)
text = re.sub(r"\s+", " ", text).strip()

print(f"\nDataset loaded. Text length: {len(text):,} characters")
print(f"Sample preview: {text[:150]}...\n")

# -------------------------2. Tokenization -------------------------
print("Tokenizing text...")
start_time = time.time()

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts([text])

total_words = min(len(tokenizer.word_index) + 1, 10001)
print(f"✓ Tokenization complete. Total unique words: {total_words}")
print(f"Time: {time.time() - start_time:.2f} seconds\n")

# ------------------------- 3. Create Sequences -------------------------
print("Creating training sequences...")
start_time = time.time()

sequences = tokenizer.texts_to_sequences([text])[0]

sequence_length = 20
input_sequences = []

for i in range(sequence_length, len(sequences)):
    seq = sequences[i-sequence_length:i+1]
    input_sequences.append(seq)

MAX_SEQUENCES = 200000
if len(input_sequences) > MAX_SEQUENCES:
    input_sequences = input_sequences[:MAX_SEQUENCES]

print(f"Created {len(input_sequences):,} training sequences (optimized)")


input_sequences = np.array(input_sequences, dtype=np.int32)
X = input_sequences[:, :-1]
y = input_sequences[:, -1]


y = tf.keras.utils.to_categorical(y, num_classes=total_words)

print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"Time: {time.time() - start_time:.2f} seconds\n")

# -------------------------4. Build Model -------------------------
print("Building optimized model...")
model = Sequential([
    Input(shape=(sequence_length,)),
    Embedding(total_words, 64),
    LSTM(128, return_sequences=True),
    LSTM(128),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(total_words, activation="softmax")
])

model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

model.summary()
print()

# ------------------------- 5. Custom Callback for Progress-------------------------
class TrainingProgress(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"\n{'='*50}")
        print(f"Epoch {epoch+1}/10")
        print(f"{'='*50}")

    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1} completed")
        print(f"Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.6f}")
        if 'val_loss' in logs:
            print(f"Val Loss: {logs['val_loss']:.4f}, Val Accuracy: {logs['val_accuracy']:.6f}")
        print(f"{'='*50}")

# ------------------------- 6. Train Model -------------------------
print("="*60)
print("TRAINING MODEL (Optimized for Speed)")
print("="*60)
print("Estimated time: 20-40 minutes (was 2-5 hours)")
print("="*60)

checkpoint = ModelCheckpoint(
    'best_shakespeare_model.keras',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    verbose=0
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True,
    verbose=1
)

progress = TrainingProgress()

start_time = time.time()
print(f"\nTraining started at: {time.strftime('%H:%M:%S')}")

history = model.fit(
    X,
    y,
    epochs=10,
    batch_size=128,
    validation_split=0.1,
    callbacks=[early_stop, checkpoint, progress],
    verbose=0
)

training_time = time.time() - start_time
print(f"\n{'='*60}")
print(f"TRAINING COMPLETED SUCCESSFULLY!")
print(f"Total training time: {training_time/60:.1f} minutes")
print(f"{'='*60}")

# ------------------------- 7. Text Generation Function -------------------------
def generate_text(seed_text, next_words=20, temperature=0.8, diversity=1.0):
    """
    Generate text starting from seed_text.

    Parameters:
    - seed_text: Starting text
    - next_words: Number of words to generate
    - temperature: Controls randomness (0.1-2.0)
    - diversity: Additional randomness factor
    """
    generated_text = seed_text

    for word_num in range(next_words):

        token_list = tokenizer.texts_to_sequences([generated_text])[0]

        if len(token_list) > sequence_length:
            token_list = token_list[-sequence_length:]


        token_list = pad_sequences(
            [token_list], maxlen=sequence_length, padding="pre"
        )

        predictions = model.predict(token_list, verbose=0)[0]

        predictions = np.log(predictions + 1e-7) / temperature
        predictions = predictions * diversity
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)

        predicted_index = np.random.choice(range(total_words), p=predictions)

        output_word = tokenizer.index_word.get(predicted_index, "")

        if output_word:
            generated_text += " " + output_word

            if word_num % 8 == 0 and word_num > 0:
                generated_text += "\n"

    return generated_text

# -------------------------8. Generate Sample Outputs-------------------------
print("\n" + "="*60)
print("GENERATING SHAKESPEARE TEXT")
print("="*60)

seed_texts = [
    "to be or not to be",
    "romeo romeo wherefore art thou",
    "all the world s a stage",
    "shall i compare thee to a",
    "now is the winter of our"
]

print("\nGenerating text samples (this may take a moment)...\n")

for i, seed in enumerate(seed_texts, 1):
    print(f"{i}. Seed: '{seed}'")
    print("-" * 50)

    generated = generate_text(seed, next_words=25, temperature=0.7)

    lines = generated.split('\n')
    for line in lines:

        words = line.strip().split()
        if words:
            words[0] = words[0].capitalize()
            print(' '.join(words))

    print()

print("\n" + "="*60)
print("SAVING MODEL AND TOKENIZER")
print("="*60)


model.save("shakespeare_lstm_optimized.keras")
print("✓ Model saved: 'shakespeare_lstm_optimized.keras'")

with open('shakespeare_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("✓ Tokenizer saved: 'shakespeare_tokenizer.pickle'")

training_history = {
    'loss': history.history['loss'],
    'accuracy': history.history['accuracy'],
    'val_loss': history.history['val_loss'],
    'val_accuracy': history.history['val_accuracy']
}

with open('training_history.pickle', 'wb') as handle:
    pickle.dump(training_history, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("✓ Training history saved: 'training_history.pickle'")


print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)

final_epoch = len(history.history['loss'])
final_loss = history.history['loss'][-1]
final_acc = history.history['accuracy'][-1]
final_val_loss = history.history['val_loss'][-1]
final_val_acc = history.history['val_accuracy'][-1]

print(f"Model Architecture: LSTM with {model.count_params():,} parameters")
print(f"Vocabulary Size: {total_words:,} words")
print(f"Training Sequences: {len(X):,}")
print(f"Sequence Length: {sequence_length} words")
print(f"Epochs Trained: {final_epoch}")
print(f"Final Training Loss: {final_loss:.4f}")
print(f"Final Training Accuracy: {final_acc:.6f}")
print(f"Final Validation Loss: {final_val_loss:.4f}")
print(f"Final Validation Accuracy: {final_val_acc:.6f}")
print(f"Total Training Time: {training_time/60:.1f} minutes")

print("\n" + "="*60)
print("HOW TO USE YOUR TRAINED MODEL")
print("="*60)
print("1. To generate more text, call:")
print("   generate_text('your seed text here', next_words=30)")
print("\n2. To load and use the saved model later:")
print("   from tensorflow.keras.models import load_model")
print("   model = load_model('shakespeare_lstm_optimized.keras')")
print("\n3. To experiment with creativity:")
print("   generate_text('to be', temperature=0.5)  # More conservative")
print("   generate_text('to be', temperature=1.5)  # More creative")

print("\n" + "="*60)
print("SCRIPT COMPLETED SUCCESSFULLY! 🎭")
print("="*60)

Loading dataset from Project Gutenberg...
✓ Dataset downloaded successfully
Using optimized dataset: 500,000 characters

Dataset loaded. Text length: 485,131 characters
Sample preview: the complete works of william shakespeare by william shakespeare contents the sonnets alls well that ends well the tragedy of antony and cleopatra as ...

Tokenizing text...
✓ Tokenization complete. Total unique words: 8325
Time: 0.05 seconds

Creating training sequences...
Created 89,829 training sequences (optimized)
X shape: (89829, 20), y shape: (89829, 8325)
Time: 0.63 seconds

Building optimized model...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 20, 64)         │       532,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 20, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8325)           │       541,125 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,312,581 (5.01 MB)

 Trainable params: 1,312,581 (5.01 MB)

 Non-trainable params: 0 (0.00 B)


TRAINING MODEL (Optimized for Speed)
Estimated time: 20-40 minutes (was 2-5 hours)

Training started at: 22:18:10

Epoch 1/10
Epoch 1 completed
Loss: 6.8673, Accuracy: 0.028313
Val Loss: 6.6107, Val Accuracy: 0.028053

Epoch 2/10
Epoch 2 completed
Loss: 6.5580, Accuracy: 0.035067
Val Loss: 6.5510, Val Accuracy: 0.041300

Epoch 3/10
Epoch 3 completed
Loss: 6.4193, Accuracy: 0.040460
Val Loss: 6.5429, Val Accuracy: 0.038517

Epoch 4/10
